## Introduction

### Pré-requis

* Ce notebook permet de peupler l'ontologie relative aux adresses définies dans le fichier lié à la variable `ont_file`.
* Il faut s'assurer que l'environnement Python qui va lancer le script possède les librairies suivantes : `os`, `sys`, `ssl`, `xml`, `rdflib`, `urllib`, `importlib`, `SPARQLWrapper`
* Avant de procéder à l'exécution du script, il faut installer deux logiciels qui devront être tournés durant le processus :
    * [Ontotext Refine](https://www.ontotext.com/products/ontotext-refine/) (ou OntoRefine) qui permet d'automatiser la conversion de données textuelles en un graphe de connaissances. Dans la section des variables globables (voir ci-dessous), deux variables sont liées au logiciel :
        * `ontorefine_cli` : chemin absolu de l'interface en ligne de commande (CLI) d'OntoRefine. Pour trouver ce chemin, il suffit de cliquer sur le bouton `Open CLI directory` présent sur l'interface de lancement du logiciel. Cela vous renvoie vers le dossier contenant la CLI dont le nom est `ontorefine-cli`.
        * `ontorefine_url` : l'URL de l'application web.  Pour trouver cette valeur, il suffit de cliquer sur le bouton `Open Refine web application` présent sur l'interface de lancement du logiciel. La valeur à associer à `ontorefine_url` est l'url à laquelle on enlève `/projects`.
    * [GraphDB](https://graphdb.ontotext.com/) qui permet de stocker et de travailler sur des graphes de connaissance. Une variable est associée au logiciel : `graphdb_url` qui est l'URL de l'application web. Le procédé est similaire à celui de `ontorefine_url` pour trouver sa valeur.

![Interface d'OntoRefine avec les boutons \`Open Refine web application\` et \`Open CLI directory\`](/../../img/interface_ontorefine.png)

### Fichiers du dossier
* `ont_file` est le fichier comprenant l'ontologie des adresses.
* `data_file` : le but du notebook est de peupler l'ontologie. Ce peuplement se fait grâce aux données de la [BAN (Base Nationale Adresse)](https://adresse.data.gouv.fr/base-adresse-nationale) dont les adresses sont stockées localement dans le fichier csv lié à la variable `data_file`.
* `mapping_file` : Afin de convertir les données présentes dans le fichier précédemment cité, un mapping est nécessaire pour les convertir en données RDF selon les recommandations de l'ontologie. Pour cela, le fichier json lié `mapping_file` existe et est utilisé par OntoRefine.
* `query_file` : À la création des données via OntoRefine, il y a des données dupliquées. Un ensemble de requêtes SPARQL détecte ces doublons et supprimer les données inutiles. Ces requêtes sont dans le fichier `query_file`.

### Présentation du processus
* Le processus est divisé en quatre parties :
    * la première fait des traitements généraux :
        * import des librairies
        * définition des variables et modification de ces dernières si nécessaire (conversion des chemins relatifs en chemins absolus pour éviter les mauvaises surprises par exemple)
        * créer de dossiers s'ils n'existent pas (`out_folder` ou `tmp_folder`).
    * la deuxième est relative à OntoRefine :
        * création d'un projet à partir du fichier lié à `data_file` ;
        * conversion en données RDF grâce au fichier `mapping_file` ;
        * export des données créées dans un graphe stocké dans le fichier `out_file` ;
        * suppression du projet.
    * la troisième est liée à l'import des données créées par OntoRefine dans GraphDB :
        * création d'un répertoire ayant `project_name` comme identifiant. S'il existe déjà, on le vide de tous ses triplets. C'est pourquoi il faut éviter de définir un identifiant qui se réfère à un répertoire déjà existant ;
        * import de l'ontologie présente dans `ont_file` et des données créées par OntoRefine qui sont dans `out_file` ;
        * nettoyage des données grâce aux requêtes présentes dans `query_file`.
    * la dernière partie permet de répondre aux questions de compétence liées au modelet adresse qui a mené à la création du fichier `ont_file`. Les réponses à ces questions sont enregistrés dans le dossier `out_folder` sous la forme des fichiers csv. Les questions sont les suivantes :
        * *Quelles sont les adresses faisant référence à un lieu dont la localisation est située dans une zone géographique donnée ?*
        * *Quelles sont les adresses faisant référence à un numéro d'immeuble situé le long d'une rue donnée ?*
        * *Quelles sont les coordonnées d'une adresse donnée (définie par son libellé) ?*

## 1. Traitements généraux

### Import des librairies

In [ ]:
import os
import sys

### Import des variables globales

In [ ]:
ont_file_name = "address_ont.ttl"
data_file_name = "addresses.csv"
query_file_name = "query.txt"
mapping_file_name = "mapping.json"
out_file_name = "addresses.ttl"
local_config_file_name = "repo_config.ttl"

out_folder_name = "out"
tmp_folder_name = "tmp"

project_name = "ban_01343"

ontorefine_url = "http://localhost:7333"
graphdb_url = "http://localhost:7200"

ontorefine_cli = "/opt/ontotext-refine/lib/app/bin/ontorefine-cli"
# ontorefine_cli = "/Applications/Ontotext\ Refine.app/Contents/app/bin/ontorefine-cli"

py_code_folder_path = "../../code"

### Traitement des variables globales

* Obtention des chemins absolus des fichiers à partir des chemins relatifs donnés dans la section précédente
* Création d'un dossier de sortie s'il n'existe pas encore pour stocker les fichiers de sortie



In [ ]:
tmp_folder = os.path.abspath(tmp_folder_name)
out_folder = os.path.abspath(out_folder_name)
python_code_folder = os.path.abspath(py_code_folder_path)

ont_file = os.path.abspath(ont_file_name)
data_file = os.path.abspath(data_file_name)
query_file = os.path.abspath(query_file_name)
mapping_file = os.path.abspath(mapping_file_name)
out_file = os.path.join(out_folder, out_file_name)
local_config_file = os.path.join(tmp_folder, local_config_file_name)

### Import des modules situés dans le dossier `code`

In [ ]:
# Appel du dossier `code` comprend les codes python
sys.path.insert(1, python_code_folder)

import graphdb as gd
import filemanagement as fm
import ontorefine as otr

### Création de dossiers s'ils n'existent pas

In [ ]:
fm.create_folder_if_not_exists(out_folder)
fm.create_folder_if_not_exists(tmp_folder)

## 2. Structuration des données contenues dans `data_file` via OntoRefine

:warning: Dans cette partie, il faut que OntoRefine soit lancé avnt d'exécuter les lignes de code.


In [ ]:
otr.get_export_file_from_ontorefine(data_file, mapping_file, out_file, ontorefine_cli, ontorefine_url, project_name)

## 3. Import des données dans GraphDB et nettoyage

:warning: Lancer GraphDB avant de lancer les commandes ci-dessous (pour que `graphdb_url` fonctionne).

### Création du répertoire local dans GraphDB
Pour que la création marche, il faut que GraphDB soit lancé et donc que l'URI donné par `graphdb_url` fonctionne. Si le répertoire existe déjà, rien n'est fait

In [ ]:
gd.create_config_local_repository_file(local_config_file, project_name)
gd.create_repository_from_config_file(graphdb_url, local_config_file)

### Vidage du répertoire local
Le répertoire dont l'id est `project_name` pour y stocker les données récupérées, cela est utile si le répertoire existait déjà.

In [ ]:
gd.clear_repository(graphdb_url, project_name)

### Import de l'ontologie et des triplets créés dans la partie liée à Ontorefine

In [ ]:
gd.import_ttl_file_in_graphdb(graphdb_url, project_name, ont_file)
gd.import_ttl_file_in_graphdb(graphdb_url, project_name, out_file)

### Nettoyage les données en supprimant les doublons

In [ ]:
# Lecture du fichier contenant la requête pour nettoyer les données
query = fm.read_file(query_file)

# Mise à jour des données grâce à la requête fournie
gd.update_query(query, graphdb_url, project_name)

## 4. Réponses aux questions de compétence

### Quelles sont les adresses faisant référence à un lieu dont la localisation est située dans une zone géographique donnée ?

In [ ]:
proj_uri = "http://www.opengis.net/def/crs/OGC/1.3/CRS84"
polygon_wkt = "POLYGON((4.957909552887547 46.2633019251873,4.965140788391698 46.2633019251873,4.965140788391698 46.258658479525806,4.957909552887547 46.258658479525806,4.957909552887547 46.2633019251873))"
out_file_name = "query1-out.csv"

query = f"""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX addr: <http://rdf.geohistoricaldata.org/address#>
PREFIX gsp: <http://www.opengis.net/ont/geosparql#>
PREFIX gspf: <http://www.opengis.net/def/function/geosparql/>

select ?s ?nb ?streetName ?coords where {{
    ?s a addr:Address; addr:targets [rdfs:label ?nb;
                       addr:isPartOf [addr:isLandmarkType addr:Thoroughfare; rdfs:label ?streetName];
                                                                              gsp:asWKT ?coords].
        FILTER (
        gspf:sfWithin(
            ?coords,
            "<{proj_uri}> {polygon_wkt}"^^gsp:wktLiteral
        )
    )

}}
"""

gd.select_query_to_txt_file(query, graphdb_url, project_name, os.path.join(out_folder,out_file_name))

### Quelles sont les adresses faisant référence à un numéro d'immeuble situé le long d'une rue donnée ?

In [ ]:
street_name = "route de l'Amitié"
lang = "fr"
out_file_name = "query2-out.csv"

query = f"""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX addr: <http://rdf.geohistoricaldata.org/address#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
SELECT ?address ?addressLabel ?geom WHERE {{
    ?lm a addr:Landmark; addr:isLandmarkType addr:Thoroughfare; rdfs:label ?streetName.
    FILTER(LCASE(STR(?streetName)) = \"{street_name.lower()}\" && LANG(?streetName) = \"{lang}\")
    ?address a addr:Address; rdfs:label ?addressLabel.
    OPTIONAL {{?address addr:targets [geo:asWKT ?geom].}}
    {{?address addr:firstStep ?addrSegment}}UNION{{?address addr:firstStep [addr:nextStep+ ?addrSegment]}}
    ?addrSegment a addr:AddressSegment; addr:relatum ?lm.
}}
"""

gd.select_query_to_txt_file(query, graphdb_url, project_name, os.path.join(out_folder,out_file_name))

### Quelles sont les coordonnées d'une adresse donnée (définie par son libellé) ?

In [ ]:
address_name = "95 route de l'Amitié, 01380 Saint-Cyr-sur-Menthon"
lang = "fr"
out_file_name = "query3-out.csv"

query = f"""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX addr: <http://rdf.geohistoricaldata.org/address#>
PREFIX gsp: <http://www.opengis.net/ont/geosparql#>

SELECT ?item ?addressLabel ?coords WHERE {{
    ?item a addr:Address;
          rdfs:label ?addressLabel; 
          addr:targets [gsp:asWKT ?coords].
    FILTER(LCASE(STR(?addressLabel)) = \"{address_name.lower()}\" && LANG(?addressLabel) = \"{lang}\")
}}
"""

gd.select_query_to_txt_file(query, graphdb_url, project_name, os.path.join(out_folder,out_file_name))